In [3]:
import nltk # type: ignore
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/sandy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sandy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sandy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/sandy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import re
import json
# The data fields are:
# text: a string feature.
# label: a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

file_path = "data.txt"
sad = []
joy = []
anger = []
surprise = []
with open(file_path, "r") as file:
    lines = file.readlines()
    for line in lines:
        if (re.search("0}$", line) != None):
            data = json.loads(line)
            text_value = data["text"]
            sad.append(text_value)
        elif (re.search("1}$", line) != None):
            data = json.loads(line)
            text_value = data["text"]
            joy.append(text_value)
        elif (re.search("3}$", line) != None):
            data = json.loads(line)
            text_value = data["text"]
            anger.append(text_value)
        elif (re.search("5}$", line) != None):
            data = json.loads(line)
            text_value = data["text"]
            surprise.append(text_value)
# print(anger[-1])

In [5]:
# Tokenization
from nltk.tokenize import word_tokenize # type: ignore

sad_tokens = []
joy_tokens = []
anger_tokens = []
surprise_tokens = []

for text in sad:
    tokens = word_tokenize(text)
    sad_tokens.append(tokens)

for text in joy:
    tokens = word_tokenize(text)
    joy_tokens.append(tokens)

for text in anger:
    tokens = word_tokenize(text)
    anger_tokens.append(tokens)

for text in surprise:
    tokens = word_tokenize(text)
    surprise_tokens.append(tokens)

# print("Original tweet:")
# print(anger[0])
# print("--------------------")
# print("Tokenized tweet:")
# print(anger_tokens[0])

In [6]:
# Normalization & Noise Removal

import string
from nltk.tag import pos_tag # type: ignore
from nltk.stem.wordnet import WordNetLemmatizer # type: ignore
from nltk.corpus import stopwords # type: ignore

stop_words = stopwords.words('english')

def remove_noise(tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

sad_tokens_preprocessed = []
joy_tokens_preprocessed = []
anger_tokens_preprocessed = []
surprise_tokens_preprocessed = []

for tokens in sad_tokens:
    sad_tokens_preprocessed.append(remove_noise(tokens, stop_words))
for tokens in joy_tokens:
    joy_tokens_preprocessed.append(remove_noise(tokens, stop_words))
for tokens in anger_tokens:
    anger_tokens_preprocessed.append(remove_noise(tokens, stop_words))
for tokens in surprise_tokens:
    surprise_tokens_preprocessed.append(remove_noise(tokens, stop_words))

print(sad_tokens_preprocessed[-1])

In [ ]:
# Building the Naive Bayes Classifier

# Convert to dictionary

def convert_dict(cleaned_tokens_list): # ! incorrect dict
    for tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tokens)

sad_tokens_dict = convert_dict(sad_tokens_preprocessed)
for keys,values in sad_tokens_dict.items():
    print(keys)
    print(values)
joy_tokens_dict = convert_dict(joy_tokens_preprocessed)
anger_tokens_dict = convert_dict(anger_tokens_preprocessed)
surprise_tokens_dict = convert_dict(surprise_tokens_preprocessed)

# Add Labels

sad_dataset = [(tweet_dict, "Sad") for tweet_dict in sad_tokens_dict]
joy_dataset = [(tweet_dict, "Joy") for tweet_dict in joy_tokens_dict]
anger_dataset = [(tweet_dict, "Anger") for tweet_dict in anger_tokens_dict]
surprise_dataset = [(tweet_dict, "Surprise") for tweet_dict in surprise_tokens_dict]

# Combine Datasets
import random
dataset = sad_dataset + joy_dataset + anger_dataset + surprise_dataset
random.shuffle(dataset)

# Split Training & Testing
train_data = dataset[:1]
test_data = dataset[7000:]

# Build the Model
from nltk import classify # type: ignore
from nltk import NaiveBayesClassifier # type: ignore
classifier = NaiveBayesClassifier.train(train_data)

# Test
print("Accuracy is:", classify.accuracy(classifier, test_data))
